# How we prepare our data for research project

# Imports

In [54]:
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
%config InlineBackend.figure_format = 'svg'

In [55]:
import warnings
warnings.filterwarnings("ignore")

# Read data

In [56]:
df = pd.read_stata('/Users/sayfulindmitry/Downloads/Working file 2015.dta')

In [57]:
df.head()

,regionx,familyx,personx,round,sitex,tid,inadult,inmoverx,inwgt_x,ixgender,...,HOHCAT_X,HHHEAD_X,LFPART_X,HASJOB_X,WNOPAY_X,FLEAVE_X,UNEM_B_X,UNEM_G_X,UNEM_S_X,UN_DUR_X
0,116,503,5,24,116,116050305,1,1,0.000000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52,93,1,24,52,52009301,1,1,0.000000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,116,6014,5,24,116,116601405,1,1,0.000000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,132,7,24,52,52013207,1,0,1.074508,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89,5023,3,24,89,89502303,1,1,0.000000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Wage

### Variables
1. Education <br>
'ixgrdlev', 'ixhighsc', 'ixhiedul' <br>
сколько, есть ли диплом, уровень <br>
50, 5000, 500 <br>
2. Age <br>
'ixbirthy' <br>
год рождения <br>
0 <br> 
3. Region <br>
'regionx' <br>
4. Gender <br>
'ixgender' <br>
5. Health <br>
'ixmiswil'  <br>
пропускали ли по болезни <br>
90
6. Official <br>
'ixpjreg', 'ixjpayof' <br>
устроен, проводка денег <br>
7300, 7000
7. Married <br>
'ixmarist' <br>
брак, 6 вариантов <br>
30
8. Experience <br>
'ixtgempy' <br>
опыт в годах <br>
2000
9. Work <br>
'ixpjoblm', 'ixhourlm' <br>
офис, часы <br>
6500, 7800
9. Salary <br>
'ixpjpayt', 'ixwagelm' <br>
ср чистая з/п в месяц за последний год; чистая з/п за последний месяц <br>
7700, 7100

In [143]:
wage_vars = ['ixgrdlev', 'ixhighsc', 'ixhiedul'] + \
            ['ixbirthy'] + ['regionx'] + ['ixgender'] + \
            ['ixmiswil'] + ['ixpjreg', 'ixjpayof', 'ixdisabl', 'ixdclass'] + \
            ['ixmarist'] + ['ixtgempy'] + ['ixpjoblm', 'ixhourlm'] + ['ixpjpayt', 'ixwagelm', 'ixinclmo'] + \
            ['ixkids', 'ixnykids']

In [144]:
wage_df = df[wage_vars]
col_rename = ['ed_years', 'ed_dip', 'grad'] + \
             ['age'] + ['fo'] + ['male'] + \
             ['ill'] + ['hired_of', 'pay_of', 'disabled', 'dis'] + \
             ['married'] + ['exp'] + ['work', 'hour_month'] + ['mean_wage', 'last_wage', 'sum_wage'] + \
             ['ixkids', 'n_kids']
wage_df.columns = col_rename
wage_df['age'] = 2015 - wage_df['age']
wage_df['age_sq'] = wage_df['age'] ** 2
wage_df['kids'] = 0
wage_df.loc[wage_df['n_kids'] > 0, 'kids'] = 1
wage_df.loc[wage_df['ixkids'] == 2, 'n_kids'] = 0
wage_df.drop(columns=['ixkids'], inplace=True)

In [145]:
wage_df.head()

,ed_years,ed_dip,grad,age,fo,male,ill,hired_of,pay_of,disabled,...,married,exp,work,hour_month,mean_wage,last_wage,sum_wage,n_kids,age_sq,kids
0,11.0,1.0,12.0,36,116,1,1.0,1.0,1.0,2.0,...,2.0,12.0,1.0,160.0,30000.0,30000.0,30000.0,2.0,1296,1
1,10.0,1.0,6.0,56,52,2,2.0,1.0,1.0,2.0,...,4.0,36.0,1.0,464.0,NaN,10000.0,16200.0,0.0,3136,0
2,9.0,NaN,5.0,20,116,1,2.0,NaN,NaN,2.0,...,2.0,99999996.0,NaN,NaN,NaN,NaN,30000.0,1.0,400,1
3,9.0,NaN,3.0,37,52,1,2.0,1.0,99999997.0,2.0,...,2.0,0.0,1.0,150.0,30000.0,27000.0,32000.0,2.0,1369,1
4,11.0,1.0,10.0,25,89,2,2.0,1.0,1.0,2.0,...,1.0,2.0,2.0,NaN,50000.0,25000.0,25000.0,0.0,625,0


In [146]:
wage_df.isna().sum()

ed_years         58
ed_dip         5629
grad            597
age               0
fo                0
male              0
ill              88
hired_of       7314
pay_of         6927
disabled          3
dis           13071
married          34
exp            2149
work           6592
hour_month     7801
mean_wage      7765
last_wage      7154
sum_wage        401
n_kids           13
age_sq            0
kids              0
dtype: int64

### Male & ill & pay_of

In [147]:
base_dict = {1: 1, 2: 0, 3: 0, 5: 1}
wage_df['male'] = wage_df['male'].map(base_dict)
wage_df['female'] = -1 * wage_df['male'] + 1
wage_df['ill'] = wage_df['ill'].map(base_dict)
wage_df['pay_of'] = wage_df['pay_of'].map(base_dict)
wage_df['work'] = wage_df['work'].map(base_dict)
wage_df['disabled'] = wage_df['disabled'].map(base_dict).fillna(0)

### Region

In [148]:
reg_list = [[1, 89, 105, 141], [9, 39, 129, 137], [52, 77], \
[10, 12, 45, 47, 48, 70, 100, 116, 117], [46, 86, 106, 107], [58, 66, 71, 73, 84, 161], [92, 93]]

In [149]:
reg_dict = {}
for i in range(len(reg_list)):
    for num in reg_list[i]:
        reg_dict[num] = i+2

In [150]:
wage_df['fo'] = wage_df['fo'].map(reg_dict)
wage_df['fo'] = wage_df['fo'].fillna(0)
wage_df['fo'] = wage_df['fo'].astype('int')

In [151]:
reg_dict2 = {0:'center', 2: 'center',
             3: 'south', 4: 'south',
             5: 'privolzh', 
            6: 'ural',
            7: 'trans-urals', 8: 'trans-urals'}
wage_df['fo'] = wage_df['fo'].map(reg_dict2)

In [152]:
wage_df.head(2)

,ed_years,ed_dip,grad,age,fo,male,ill,hired_of,pay_of,disabled,...,exp,work,hour_month,mean_wage,last_wage,sum_wage,n_kids,age_sq,kids,female
0,11.0,1.0,12.0,36,privolzh,1,1.0,1.0,1.0,0.0,...,12.0,1.0,160.0,30000.0,30000.0,30000.0,2.0,1296,1,0
1,10.0,1.0,6.0,56,south,0,0.0,1.0,1.0,0.0,...,36.0,1.0,464.0,NaN,10000.0,16200.0,0.0,3136,0,1


### Education

In [153]:
# 8, 13, 14 - drop it

educ_dict = {1: 'sch', 2: 'sch', 3: 'tech1', 4: 'tech1', 5: 'tech1', 6: 'tech2', \
            10: 'uni', 11: 'uni', 12: 'uni', \
            8: 'asp', \
            13: 'asp', 14: 'asp'}
wage_df['grad'] = wage_df['grad'].map(educ_dict)
# wage_df.loc[wage_df['grad'] != 'asp'].shape, wage_df.shape
wage_df = wage_df.loc[wage_df['grad'] != 'asp'].copy()

### Marriage

In [154]:
marr_dict = {1: 0, 4: 0, 5: 0, 2: 1, 3: 1, 6: 1}
wage_df['married'] = wage_df['married'].map(marr_dict)
#wage_df['fem_mar'] = wage_df['married'] * wage_df['female']

### Wage

In [155]:
wage_df.loc[wage_df['sum_wage'] > 0, 'last_wage'] = wage_df.loc[wage_df['sum_wage'] > 0, 'last_wage'].fillna(0)
wage_df.loc[wage_df['last_wage']==0, 'hour_month'] = wage_df.loc[wage_df['last_wage']==0, 'hour_month'].fillna(0)
wage_df.loc[wage_df['last_wage']==0, 'work'] = wage_df.loc[wage_df['last_wage']==0, 'work'].fillna(0)

In [156]:
wage_df.sample(3)

,ed_years,ed_dip,grad,age,fo,male,ill,hired_of,pay_of,disabled,...,exp,work,hour_month,mean_wage,last_wage,sum_wage,n_kids,age_sq,kids,female
10741,9.0,NaN,NaN,15,privolzh,0,0.0,NaN,NaN,0.0,...,NaN,0.0,0.0,NaN,0.0,300.0,0.0,225,0,1
10770,8.0,NaN,sch,66,center,1,0.0,NaN,NaN,0.0,...,36.0,0.0,0.0,NaN,0.0,7400.0,0.0,4356,0,0
9222,7.0,NaN,sch,27,trans-urals,1,0.0,NaN,NaN,0.0,...,NaN,0.0,0.0,NaN,0.0,25000.0,1.0,729,1,0


In [157]:
wage_df['rest_wage'] = wage_df['sum_wage'] - wage_df['last_wage']

In [158]:
wage_df[['rest_wage']].sort_values('rest_wage', ascending=False).head()

,rest_wage
9443,1218000.0
9579,1214300.0
2822,450000.0
5823,424000.0
9616,384300.0


In [159]:
wage_df.isna().sum()

ed_years         56
ed_dip         5624
grad            597
age               0
fo                0
male              0
ill              88
hired_of       7290
pay_of         7252
disabled          0
dis           13001
married          34
exp            2144
work           1508
hour_month     2612
mean_wage      7737
last_wage      1884
sum_wage        396
n_kids           13
age_sq            0
kids              0
female            0
rest_wage      1907
dtype: int64

In [160]:
wage_df['sum_wage'].sort_values(ascending=False).head()

9443    1226000.0
9579    1214300.0
2822     610000.0
5823     424000.0
9616     402300.0
Name: sum_wage, dtype: float64

### Disabled

In [161]:
wage_df['dis'] = wage_df['dis'].fillna(0)

In [162]:
wage_df.head()

,ed_years,ed_dip,grad,age,fo,male,ill,hired_of,pay_of,disabled,...,work,hour_month,mean_wage,last_wage,sum_wage,n_kids,age_sq,kids,female,rest_wage
0,11.0,1.0,uni,36,privolzh,1,1.0,1.0,1.0,0.0,...,1.0,160.0,30000.0,30000.0,30000.0,2.0,1296,1,0,0.0
1,10.0,1.0,tech2,56,south,0,0.0,1.0,1.0,0.0,...,1.0,464.0,NaN,10000.0,16200.0,0.0,3136,0,1,6200.0
2,9.0,NaN,tech1,20,privolzh,1,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0.0,30000.0,1.0,400,1,0,30000.0
3,9.0,NaN,tech1,37,south,1,0.0,1.0,NaN,0.0,...,1.0,150.0,30000.0,27000.0,32000.0,2.0,1369,1,0,5000.0
4,11.0,1.0,uni,25,center,0,0.0,1.0,1.0,0.0,...,0.0,NaN,50000.0,25000.0,25000.0,0.0,625,0,1,0.0


# To drop

In [163]:
columns_to_drop = ['ed_years', 'ed_dip', 'hired_of', 'pay_of', 'mean_wage']
wage_df.drop(columns=columns_to_drop, inplace=True)

# Save df (and drop)

In [164]:
wage_df = wage_df.loc[(wage_df['age'] <= 75) & (wage_df['age'] >= 14) & (wage_df['exp'] < 100)].dropna().copy()
# .reset_index(drop=True)

##### fem_mar

In [165]:
wage_df['fem_mar'] = (wage_df['married'] * wage_df['female']).astype('int')

In [166]:
wage_df.shape

(9079, 19)

# Do it after drop

In [167]:
wage_df['last_wage_hour'] = (wage_df['last_wage'] / wage_df['hour_month']).round(1).fillna(0)

In [168]:
wage_df = wage_df.replace([np.inf, -np.inf], 0)

In [181]:
wage_df.isna().sum()

age               0
male              0
ill               0
disabled          0
married           0
exp               0
work              0
hour_month        0
last_wage         0
sum_wage          0
n_kids            0
age_sq            0
kids              0
female            0
nl_income         0
fem_mar           0
last_wage_hour    0
grad_sch          0
grad_tech1        0
grad_tech2        0
grad_uni          0
fo_privolzh       0
fo_south          0
fo_trans-urals    0
fo_ural           0
dis_1             0
dis_2             0
dis_3             0
dtype: int64

last wage, woork==0 => fillna(0)

In [170]:
wage_df['sum_wage'].sort_values(ascending=False).head()

9443    1226000.0
9579    1214300.0
2822     610000.0
5823     424000.0
9616     402300.0
Name: sum_wage, dtype: float64

# OHE

In [171]:
wage_df['fem_mar'] = wage_df['married'] * wage_df['female']
wage_df[['male', 'ill', 'work', 'married', 'fem_mar', 'disabled', 'dis']] =  \
wage_df[['male', 'ill', 'work', 'married', 'fem_mar', 'disabled', 'dis']].astype('int')

In [172]:
wage_df = pd.get_dummies(wage_df, columns=['grad', 'fo', 'dis'])

In [173]:
wage_df.sample(2)

,age,male,ill,disabled,married,exp,work,hour_month,last_wage,sum_wage,...,grad_uni,fo_center,fo_privolzh,fo_south,fo_trans-urals,fo_ural,dis_0,dis_1,dis_2,dis_3
13933,37,1,0,0,1,18.0,1,176.0,21000.0,21000.0,...,0,1,0,0,0,0,1,0,0,0
5221,28,1,0,0,1,5.0,1,192.0,30000.0,30000.0,...,0,1,0,0,0,0,1,0,0,0


In [174]:
wage_df.drop(columns=['fo_center', 'dis_0'], inplace=True)

In [175]:
wage_df = wage_df.rename(columns={'rest_wage': 'nl_income'})

In [253]:
#wage_df.columns =['age', 'male', 'ill', 'disabled', 'married', 'exp', 'work',
#                  'hour_month', 'last_wage', 'sum_wage', 'age_sq', 'female', 'nl_income', 
#                  'fem_mar','wage', 'grad_asp', 'grad_sch', 'grad_sci', 'grad_uni',
#                  'privolzh', 'south', 'trans-urals', 'ural', 'grad_phd']

In [176]:
wage_df.columns

Index(['age', 'male', 'ill', 'disabled', 'married', 'exp', 'work',
       'hour_month', 'last_wage', 'sum_wage', 'n_kids', 'age_sq', 'kids',
       'female', 'nl_income', 'fem_mar', 'last_wage_hour', 'grad_sch',
       'grad_tech1', 'grad_tech2', 'grad_uni', 'fo_privolzh', 'fo_south',
       'fo_trans-urals', 'fo_ural', 'dis_1', 'dis_2', 'dis_3'],
      dtype='object')

In [177]:
wage_df.loc[(wage_df['disabled'] == 1) & (wage_df['dis_1'] == 0) & (wage_df['dis_2'] == 0) & (wage_df['dis_3'] == 0)]

,age,male,ill,disabled,married,exp,work,hour_month,last_wage,sum_wage,...,grad_tech1,grad_tech2,grad_uni,fo_privolzh,fo_south,fo_trans-urals,fo_ural,dis_1,dis_2,dis_3
2117,70,0,0,1,0,36.0,0,0.0,0.0,12200.0,...,1,0,0,0,0,0,1,0,0,0
2818,73,1,0,1,1,47.0,0,0.0,0.0,14000.0,...,0,1,0,0,0,0,0,0,0,0
5138,63,0,0,1,0,40.0,0,0.0,0.0,11300.0,...,0,1,0,0,1,0,0,0,0,0
7333,68,1,0,1,1,33.0,0,0.0,0.0,17000.0,...,0,1,0,0,0,1,0,0,0,0
8758,58,0,0,1,1,36.0,0,0.0,0.0,11100.0,...,0,1,0,0,0,0,0,0,0,0
11098,67,0,1,1,0,45.0,0,0.0,0.0,20500.0,...,0,0,0,1,0,0,0,0,0,0
11313,46,0,0,1,1,12.0,0,0.0,0.0,6100.0,...,0,0,0,0,1,0,0,0,0,0


# Save df for project

In [178]:
wage_df.to_excel('wage_data_8.xlsx', index=False)

In [179]:
wage_df.shape

(9079, 28)

In [180]:
wage_df.dtypes

age                 int64
male                int64
ill                 int64
disabled            int64
married             int64
exp               float64
work                int64
hour_month        float64
last_wage         float64
sum_wage          float64
n_kids            float64
age_sq              int64
kids                int64
female              int64
nl_income         float64
fem_mar             int64
last_wage_hour    float64
grad_sch            uint8
grad_tech1          uint8
grad_tech2          uint8
grad_uni            uint8
fo_privolzh         uint8
fo_south            uint8
fo_trans-urals      uint8
fo_ural             uint8
dis_1               uint8
dis_2               uint8
dis_3               uint8
dtype: object

_______

_____

# Unique families

In [182]:
df['code'] = (df['regionx'].astype('str') + df['familyx'].astype('str')).astype('int')

In [183]:
df['code'].nunique()

6355

In [185]:
un_dict = {}
for i in range(len(df['code'].unique())):
    un_dict[df['code'].unique()[i]] = i+1    

In [186]:
df['code'] = df['code'].map(un_dict)

### Amount of members

In [100]:
am_df = df[['code', 'personx']].groupby('code', as_index=False).max()
am_dict = {}
for i in range(am_df.shape[0]):
    am_dict[am_df['code'][i]] = am_df['personx'][i]

In [101]:
df['member'] = df['code'].map(am_dict)

In [187]:
df.sample(3)

,regionx,familyx,personx,round,sitex,tid,inadult,inmoverx,inwgt_x,ixgender,...,FLEAVE_X,UNEM_B_X,UNEM_G_X,UNEM_S_X,UN_DUR_X,code,member,sen_member,mid_member,kid_sum
657,71,174,2,24,71,71017402,1,0,0.700037,2,...,NaN,NaN,NaN,NaN,NaN,622,4,1,3,0
10114,1,237,5,24,1,1023705,1,1,0.000000,1,...,0.0,0.0,0.0,0.0,NaN,5310,6,0,9,0
9857,58,6,10,24,58,58000610,1,1,0.000000,1,...,0.0,0.0,0.0,0.0,NaN,5147,10,0,3,0


### Senior people

In [191]:
sen_df = df.loc[df['ixbirthy'] < 1950, ['code', 'ixbirthy']].groupby('code', as_index=False).count()
sen_dict = {}
for i in range(sen_df.shape[0]):
    sen_dict[sen_df['code'][i]] = sen_df['ixbirthy'][i]

In [192]:
df['sen_member'] = df['code'].map(sen_dict)
df['sen_member'] = df['sen_member'].fillna(0).astype('int')

### Middle

In [193]:
mid_df = df.loc[df['ixbirthy'] >= 1950, ['code', 'ixbirthy']].groupby('code', as_index=False).count()
mid_dict = {}
for i in range(mid_df.shape[0]):
    mid_dict[mid_df['code'][i]] = mid_df['ixbirthy'][i]

In [194]:
df['mid_member'] = df['code'].map(mid_dict)
df['mid_member'] = df['mid_member'].fillna(0).astype('int')

In [195]:
df.sample(3)

,regionx,familyx,personx,round,sitex,tid,inadult,inmoverx,inwgt_x,ixgender,...,FLEAVE_X,UNEM_B_X,UNEM_G_X,UNEM_S_X,UN_DUR_X,code,member,sen_member,mid_member,kid_sum
10505,14,61,2,24,16,14006102,1,0,0.935943,2,...,0.0,0.0,0.0,0.0,NaN,5527,2,0,2,0
1812,100,94,1,24,100,100009401,1,0,0.700037,2,...,NaN,NaN,NaN,NaN,NaN,1577,1,1,0,0
12323,71,125,4,24,71,71012504,1,0,1.286979,1,...,0.0,0.0,0.0,0.0,NaN,5887,4,0,3,0


### Kids

In [108]:
kid_df = df[['code', 'ixnykids']].groupby('code', as_index=False).sum()
kid_dict = {}
for i in range(kid_df.shape[0]):
    kid_dict[kid_df['code'][i]] = kid_df['ixnykids'][i]

In [109]:
df['kid_sum'] = df['code'].map(kid_dict)
df['kid_sum'] = df['kid_sum'].fillna(0).astype('int')

In [192]:
df['kid'] = np.ceil(df['kid_sum'] / df['mid_member']).fillna(0).astype('int')

In [193]:
df.loc[df.mid_member ==0, ['code', 'member','sen_member', 'mid_member', 'kid_sum', 'kid']].head(2)

,code,member,sen_member,mid_member,kid_sum,kid
23,24,2,2,0,0,0
28,28,2,2,0,0,0


# Merge df

In [196]:
columns_to_merge = ['code', 'member', 'sen_member']

In [261]:
#est_wages = pd.read_csv('wage_est_xlsx.csv') #wait for Max's wages

In [300]:
est_wages.loc[est_wages['wage_1'] < 0, 'wage_1'] = 0
est_wages.loc[est_wages['wage_2'] < 0, 'wage_2'] = 0
est_wages.loc[est_wages['wage_3'] < 0, 'wage_3'] = 0
est_wages.loc[est_wages['wage_4'] < 0, 'wage_4'] = 0

In [197]:
family_df = wage_df.join(df[columns_to_merge])
family_df.reset_index(drop=True, inplace=True)
#family_df = family_df.join(est_wages)

In [311]:
family_df = family_df.join(est_wages)

In [312]:
family_df.shape, est_wages.shape

((9130, 34), (9130, 4))

In [199]:
family_df.columns

Index(['age', 'male', 'ill', 'disabled', 'married', 'exp', 'work',
       'hour_month', 'last_wage', 'sum_wage', 'n_kids', 'age_sq', 'kids',
       'female', 'nl_income', 'fem_mar', 'last_wage_hour', 'grad_sch',
       'grad_tech1', 'grad_tech2', 'grad_uni', 'fo_privolzh', 'fo_south',
       'fo_trans-urals', 'fo_ural', 'dis_1', 'dis_2', 'dis_3', 'code',
       'member', 'sen_member'],
      dtype='object')

# Aggregate df

In [204]:
columns_to_agg = columns_to_merge

In [205]:
family_df.columns

Index(['age', 'male', 'ill', 'disabled', 'married', 'exp', 'work',
       'hour_month', 'last_wage', 'sum_wage', 'n_kids', 'age_sq', 'kids',
       'female', 'nl_income', 'fem_mar', 'last_wage_hour', 'grad_sch',
       'grad_tech1', 'grad_tech2', 'grad_uni', 'fo_privolzh', 'fo_south',
       'fo_trans-urals', 'fo_ural', 'dis_1', 'dis_2', 'dis_3', 'code',
       'member', 'sen_member'],
      dtype='object')

In [206]:
final_df = family_df[columns_to_agg].groupby('code', as_index=False).max()
final_df = final_df.join(family_df[['code', 'hour_month', 'disabled', 'nl_income']] \
                         .groupby('code', as_index=False).sum().iloc[:, 1:])

In [321]:
final_df = final_df.join(family_df[['code'] + list(est_wages.columns)] \
                         .groupby('code', as_index=False).sum().iloc[:, 1:])

### Dict for disabled

In [208]:
final_df['disabled'].value_counts()

0    4510
1     683
2      58
3       1
4       1
Name: disabled, dtype: int64

In [211]:
final_df['code'][1]

2

In [212]:
dis_dict = {}
for i in range(final_df.shape[0]):
    dis_dict[final_df['code'][i]] = final_df['disabled'][i]


In [215]:
family_df['n_dis'] = family_df['code'].map(dis_dict)

In [218]:
family_df['n_dis'].value_counts()

0    7683
1    1250
2     139
4       4
3       3
Name: n_dis, dtype: int64

In [219]:
family_df.isna().sum()

age               0
male              0
ill               0
disabled          0
married           0
exp               0
work              0
hour_month        0
last_wage         0
sum_wage          0
n_kids            0
age_sq            0
kids              0
female            0
nl_income         0
fem_mar           0
last_wage_hour    0
grad_sch          0
grad_tech1        0
grad_tech2        0
grad_uni          0
fo_privolzh       0
fo_south          0
fo_trans-urals    0
fo_ural           0
dis_1             0
dis_2             0
dis_3             0
code              0
member            0
sen_member        0
n_dis             0
dtype: int64

# Save family df

In [222]:
columns_fam = ['age', 'age_sq', 'female', 'married', 'fem_mar', 
              'exp', 'n_kids', 'kids', 'grad_sch', 'grad_tech1', 'grad_tech2', 'grad_uni',
              'fo_privolzh', 'fo_south', 'fo_trans-urals', 'fo_ural',
              'dis_1', 'dis_2', 'dis_3',
              'member', 'sen_member', 'n_dis']

In [224]:
family_df[columns_fam].to_excel('family_df1.xlsx', index=False)

In [221]:
family_df.columns

Index(['age', 'male', 'ill', 'disabled', 'married', 'exp', 'work',
       'hour_month', 'last_wage', 'sum_wage', 'n_kids', 'age_sq', 'kids',
       'female', 'nl_income', 'fem_mar', 'last_wage_hour', 'grad_sch',
       'grad_tech1', 'grad_tech2', 'grad_uni', 'fo_privolzh', 'fo_south',
       'fo_trans-urals', 'fo_ural', 'dis_1', 'dis_2', 'dis_3', 'code',
       'member', 'sen_member', 'n_dis'],
      dtype='object')

In [225]:
family_df[columns_fam].shape

(9079, 22)

#### For wages

In [579]:
columns_wage = [col for col in family_df.columns if 'wage' in col]
columns_wage = columns_wage[3:] + ['code']

In [584]:
final_df = final_df.join(family_df[columns_wage].groupby('code', as_index=False).sum().iloc[:, 1:])

# Save df for project

##### shape names

In [596]:
final_df.columns = ['code', 'member', 'sen_member', 'mid_member', 'kid_sum', 'kid', 
                    'privolzh', 'south', 'trans-urals', 'ural', 
                    'wage1', 'wage2', 'wage4', 'wage3', 'wage5', 'sum_hour']

In [295]:
final_df.rename(columns = {'rest_wage': 'nl_income'}, inplace=True)

In [323]:
final_df.to_excel('final_df5.xlsx', index=False)

In [298]:
final_df.isna().sum()

code           0
member         0
sen_member     0
mid_member     0
kid_sum        0
kid            0
privolzh       0
south          0
trans-urals    0
ural           0
hour_month     0
disabled       0
nl_income      0
wage_1         0
wage_2         0
wage_3         0
wage_4         0
dtype: int64

In [226]:
fam_df = pd.read_excel('family_df2_2.xlsx')

In [249]:
fam_df[['nl_income', 'hour_month', 'disabled']] = family_df[['nl_income', 'hour_month', 'disabled']]
fam_df['sen_member_bin'] = 0
fam_df.loc[fam_df['sen_member'] > 0, 'sen_member_bin'] = 1
fam_df['dis_bin'] = 0
fam_df.loc[fam_df['disabled'] > 0, 'dis_bin'] = 1
fam_df['nl_income_th'] = fam_df['nl_income'] / 1000

fam_df['sen_member_65'] = 0
fam_df.loc[fam_df['age'] > 65, 'sen_member_65'] = 1
fam_df['sen_member_others'] = fam_df['sen_member'] - fam_df['sen_member_65']

fam_df['n_dis_others'] = fam_df['n_dis'] - fam_df['disabled']

In [242]:
fam_df.rename(columns={'fo_privolzh': 'privolzh', 'fo_south': 'south',
                       'fo_trans-urals': 'trans-urals', 'fo_ural': 'ural'}, inplace=True)

In [243]:
fam_df.columns

Index(['age', 'age_sq', 'female', 'married', 'fem_mar', 'exp', 'n_kids',
       'kids', 'grad_sch', 'grad_tech1', 'grad_tech2', 'grad_uni', 'privolzh',
       'south', 'trans-urals', 'ural', 'dis_1', 'dis_2', 'dis_3', 'member',
       'sen_member', 'n_dis', 'wage1', 'wage2', 'nl_income', 'hour_month',
       'disabled', 'sen_member_bin', 'dis_bin', 'nl_income_th'],
      dtype='object')

In [253]:
fam_df.to_excel('family_df5.xlsx', index=False)

In [234]:
24 * 30

720

In [235]:
fam_df['hour_month'].max()

890.0

In [246]:
fam_df.shape

(9079, 33)

In [252]:
fam_df['n_dis_others'].value_counts()

0    8366
1     683
2      26
3       4
Name: n_dis_others, dtype: int64

In [232]:
pd.read_excel('family_df2_2.xlsx').shape

(9079, 24)

# Try to fill negative to 0

In [322]:
final_df[final_df['wage_1'] < 0]

,code,member,sen_member,mid_member,kid_sum,kid,privolzh,south,trans-urals,ural,hour_month,disabled,nl_income,wage_1,wage_2,wage_3,wage_4


In [279]:
est_wages.sort_values('wage_1', ascending=False)[est_wages['wage_4'] < 0]

,wage_1,wage_2,wage_3,wage_4
643,66.608016,67.217689,-1.020309,-0.850156
2009,57.154907,58.344526,-4.635913,-3.771101
7237,54.971314,59.051282,-6.959466,-5.073329
3768,54.503779,57.056393,-3.283450,-2.007872
4725,48.494866,49.774573,-7.245049,-6.498360
5631,44.041070,44.888228,-8.612843,-8.676139
1737,39.857926,42.057312,-11.919008,-10.660807
469,36.783422,38.837918,-9.679812,-8.784891
3586,36.783422,38.837918,-4.191353,-3.363493
8995,36.651346,40.923912,-20.384683,-18.603933
